# VIDEO EMOTION DETECTION


In [ ]:
import numpy as np
import argparse
import matplotlib.pyplot as plt


In [ ]:
conda info --envs

# conda environments:
#
base                  *  /opt/anaconda3


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D


In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
pip install opencv-python

  Using cached opencv-python-4.4.0.44.tar.gz (88.9 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for opencv-python: filename=opencv_python-4.4.0.44-cp38-cp38-macosx_10_12_x86_64.whl size=25642685 sha256=b1db892dd3ca04c4ecf9ef12110e0388dd2b6ec318d1759c338e1d2117ff9a46
  Stored in directory: /Users/home/Library/Caches/pip/wheels/8f/2e/c4/3f1be5b275ea88b7ad1fb92991709bba0da241c06a6bca58a3
Successfully built opencv-python
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2

In [ ]:
import sys, os  
import pandas as pd  
import numpy as np  

In [ ]:
from keras.models import Sequential  
from keras.layers import Dense, Dropout, Activation, Flatten  
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D  
from keras.losses import categorical_crossentropy  
from keras.optimizers import Adam  
from keras.regularizers import l2  
from keras.utils import np_utils  

In [ ]:
# pd.set_option('display.max_rows', 500)  
# pd.set_option('display.max_columns', 500)  
# pd.set_option('display.width', 1000)  
  
df=pd.read_csv('fer2013.csv') 

In [ ]:
df

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
...,...,...,...
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest


In [ ]:
# print(df.info())  
# print(df["Usage"].value_counts())  
  
# print(df.head())  
X_train,train_y,X_test,test_y=[],[],[],[]  
  

In [ ]:
for index, row in df.iterrows():  
    val=row['pixels'].split(" ")  
    try:  
        if 'Training' in row['Usage']:  
           X_train.append(np.array(val,'float32'))  
           train_y.append(row['emotion'])  
        elif 'PublicTest' in row['Usage']:  
           X_test.append(np.array(val,'float32'))  
           test_y.append(row['emotion'])  
    except:  
        print(f"error occured at index :{index} and row:{row}")  

In [ ]:
num_features = 64  
num_labels = 7  
batch_size = 64  
epochs = 30  
width, height = 48, 48  

In [ ]:
X_train = np.array(X_train,'float32')  
train_y = np.array(train_y,'float32')  
X_test = np.array(X_test,'float32')  
test_y = np.array(test_y,'float32')  

In [ ]:
train_y=np_utils.to_categorical(train_y, num_classes=num_labels)  
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)  

In [ ]:
#cannot produce  
#normalizing data between oand 1  
X_train -= np.mean(X_train, axis=0)  
X_train /= np.std(X_train, axis=0)  
  
X_test -= np.mean(X_test, axis=0)  
X_test /= np.std(X_test, axis=0)  
  
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)  
  
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)  

In [ ]:
# print(f"shape:{X_train.shape}")  
##designing the cnn  
#1st convolution layer  
model = Sequential()  

In [ ]:
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))  
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))  
# model.add(BatchNormalization())  
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))  
model.add(Dropout(0.5))  
  

In [ ]:
#2nd convolution layer  
model.add(Conv2D(64, (3, 3), activation='relu'))  
model.add(Conv2D(64, (3, 3), activation='relu'))  
# model.add(BatchNormalization())  
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))  
model.add(Dropout(0.5))  

In [ ]:
#3rd convolution layer  
model.add(Conv2D(128, (3, 3), activation='relu'))  
model.add(Conv2D(128, (3, 3), activation='relu'))  
# model.add(BatchNormalization())  
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))  
  
model.add(Flatten())  

In [ ]:
#fully connected neural networks  
model.add(Dense(1024, activation='relu'))  
model.add(Dropout(0.2))  
model.add(Dense(1024, activation='relu'))  
model.add(Dropout(0.2))  
  
model.add(Dense(num_labels, activation='softmax'))  
  

In [ ]:
#Compliling the model  
model.compile(loss=categorical_crossentropy,  
              optimizer=Adam(),  
              metrics=['accuracy'])  

In [ ]:
model.fit(X_train, train_y,  
          batch_size=batch_size,  
          epochs=epochs,  
          verbose=1,  
          validation_data=(X_test, test_y),  
          shuffle=True)  
  

Epoch 1/30
449/449 [==============================] - 917s 2s/step - loss: 1.7272 - accuracy: 0.2924 - val_loss: 1.5372 - val_accuracy: 0.3845
Epoch 2/30
449/449 [==============================] - 1023s 2s/step - loss: 1.5098 - accuracy: 0.4059 - val_loss: 1.4034 - val_accuracy: 0.4611
Epoch 3/30
449/449 [==============================] - 725s 2s/step - loss: 1.4074 - accuracy: 0.4545 - val_loss: 1.3195 - val_accuracy: 0.4837
Epoch 4/30
449/449 [==============================] - 740s 2s/step - loss: 1.3453 - accuracy: 0.4780 - val_loss: 1.3127 - val_accuracy: 0.4834
Epoch 5/30
449/449 [==============================] - 746s 2s/step - loss: 1.2990 - accuracy: 0.4976 - val_loss: 1.2874 - val_accuracy: 0.4999
Epoch 6/30
449/449 [==============================] - 578s 1s/step - loss: 1.2564 - accuracy: 0.5172 - val_loss: 1.2614 - val_accuracy: 0.5249
Epoch 7/30
449/449 [==============================] - 1238s 3s/step - loss: 1.2295 - accuracy: 0.5264 - val_loss: 1.1956 - val_accuracy: 0.53

In [ ]:
#Saving the  model to  use it later on  
fer_json = model.to_json()  
with open("fer.json", "w") as json_file:  
    json_file.write(fer_json)  
model.save_weights("fer.h5")  

In [ ]:
import os  
import cv2  
import numpy as np  
from keras.models import model_from_json  
from keras.preprocessing import image  
  

In [ ]:
  
#load model  
model = model_from_json(open("fer.json", "r").read())  
#load weights  
model.load_weights('fer.h5')  

In [ ]:
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
cap=cv2.VideoCapture(0)  
  

In [ ]:
while True:  
    ret,test_img=cap.read()# captures frame and returns boolean value and captured image  
    if not ret:  
        continue  
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)  
  
    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5) 
    
    for (x,y,w,h) in faces_detected:  
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)  
        roi_gray=gray_img[y:y+w,x:x+h]#cropping region of interest i.e. face area from  image  
        roi_gray=cv2.resize(roi_gray,(48,48))  
        img_pixels = image.img_to_array(roi_gray)  
        img_pixels = np.expand_dims(img_pixels, axis = 0)  
        img_pixels /= 255  
  
        predictions = model.predict(img_pixels)  
  
        #find max indexed array  
        max_index = np.argmax(predictions[0])  
  
        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')  
        predicted_emotion = emotions[max_index]  
  
        cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)  
  
    resized_img = cv2.resize(test_img, (1000, 700))  
    cv2.imshow('Facial emotion analysis ',resized_img)  
  
  
  
    if cv2.waitKey(10) == ord('q'):#wait until 'q' key is pressed  
        break  
  
cap.release()  
cv2.destroyAllWindows  

<function destroyAllWindows>